# LLM+RAG Resume Q&A Demo

[GitHub Repository](https://github.com/cool112624/llm-rag-resume)

This notebook lets you upload a resume (PDF or TXT) and interactively ask questions about its content,
using Sentence Transformers + FAISS for retrieval and OpenAI GPT-4o mini for answer generation.

**No API keys or personal data are stored in this notebook.**

In [ ]:
# 1. Install dependencies
!pip install sentence-transformers faiss-cpu openai pdfplumber

In [ ]:
# 2. Upload your resume (PDF or TXT)
print("⬆️ Please upload your resume (PDF or TXT) using the button below or by dragging and dropping:")

from google.colab import files
uploaded = files.upload()

In [ ]:
# 3. Extract text from resume (if PDF)
import os
resume_file = next(iter(uploaded))
resume_text = ""
if resume_file.lower().endswith('.pdf'):
    import pdfplumber
    with pdfplumber.open(resume_file) as pdf:
        resume_text = "\n".join(page.extract_text() for page in pdf.pages)
elif resume_file.lower().endswith('.txt'):
    with open(resume_file, encoding="utf-8") as f:
        resume_text = f.read()
else:
    raise ValueError("Please upload a PDF or TXT file.")

print(resume_text[:2000])  # Preview

In [ ]:
# 4. Split resume into chunks (by paragraphs)
def parse_resume_by_paragraph(text: str, min_length=40) -> list:
    # Split on double newlines and filter short chunks
    return [p.strip() for p in text.split('\n\n') if len(p.strip()) >= min_length]

resume_chunks = parse_resume_by_paragraph(resume_text)
print(f"Parsed {len(resume_chunks)} chunks. Example:", resume_chunks[:2])

In [ ]:
# 5. Generate embeddings and build FAISS index
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(resume_chunks)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

In [ ]:
# 6. API key input (never stored)
import openai
from getpass import getpass

api_key = getpass("Enter your OpenAI API key: ")

In [ ]:
# 7. Q&A Function
client = openai.OpenAI(api_key=api_key)
def rag_qa(query, top_k=3):
    query_emb = embedder.encode([query])
    _, I = index.search(np.array(query_emb), top_k)
    context = "\n\n".join([resume_chunks[i] for i in I[0]])
    prompt = f"Given the following context from my resume:\n\n{context}\n\nAnswer this question: {query}\n"
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


In [ ]:
# 8. Interactive Q&A Loop
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Multi-line chat input
q_box = widgets.Textarea(
    value='',
    placeholder='Type your question here (type exit or goodbye to quit)...',
    layout=widgets.Layout(width='100%', height='60px')
)
ask_button = widgets.Button(description="Send", button_style="primary")

# Scrollable chat history area
chat_output = widgets.Output(layout={
    'border': '1px solid #ccc',
    'height': '350px',
    'overflow_y': 'auto',
    'padding': '8px',
    'background': '#fafbfc'
})
chat_history = []

def show_chat():
    chat_output.clear_output()
    with chat_output:
        for entry in chat_history:
            # User block
            display(Markdown(
                f"""<div style="background:#f0f4ff;padding:8px 12px;border-radius:8px;margin-bottom:3px;">
                <b>🧑 You:</b><br>{entry['question']}
                </div>"""
            ))
            display(Markdown("<br>"))
            # AI block
            display(Markdown(
                f"""<div style="background:#f8fff0;padding:8px 12px;border-radius:8px;margin-bottom:14px;">
                <b>🤖 AI:</b><br>{entry['answer']}
                </div><br>"""   # <-- this <br> adds a blank row after AI answer
            ))

def on_ask_clicked(b):
    question = q_box.value.strip()
    if not question:
        return
    if question.lower() in ('exit', 'goodbye'):
        chat_history.append({'question': question, 'answer': "Goodbye! Chat ended. Thank you for using LLM-RAG Resume Q&A."})
        show_chat()
        q_box.disabled = True
        ask_button.disabled = True
        return
    answer = rag_qa(question)
    chat_history.append({'question': question, 'answer': answer})
    show_chat()
    q_box.value = ''  # Clear input

ask_button.on_click(on_ask_clicked)

display(widgets.VBox([
    widgets.HTML("<h3>🤖 LLM-RAG Resume Chat</h3><p>Ask anything about your uploaded resume below! Type <b>exit</b> or <b>goodbye</b> to end the chat.</p>"),
    chat_output,
    q_box,
    ask_button
]))

show_chat()